In [1]:
import os

# The Google CloudNotebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
!pip -q install {USER_FLAG} --upgrade transformers
!pip -q install {USER_FLAG} --upgrade datasets
!pip -q install {USER_FLAG} --upgrade tqdm
!pip -q install {USER_FLAG} --upgrade cloudml-hypertune
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
PROJECT_ID = "awesome-ridge-333421"  # <---CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "awesome-ridge-333421":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

Project ID:  awesome-ridge-333421
Please set your project id before proceeding to next step. Currently it's set as awesome-ridge-333421


In [2]:
from datetime import datetime


def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")


TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

TIMESTAMP = 20211204165919


In [3]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [4]:
BUCKET_NAME = "gs://story-gen2"  # <---CHANGE THIS TO YOUR BUCKET
REGION = "us-central1"
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_NAME = {BUCKET_NAME}")
print(f"REGION = {REGION}")
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-xla.1-7:latest"
)
deploy_image=("us-docker.pkg.dev/vertex-ai/prediction/pytorch-xla.1-9:latest")
PYTHON_PACKAGE_APPLICATION_DIR = "python_package"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = (
    f"{BUCKET_NAME}/story2/python_package/dist/trainer-0.1.tar.gz"
)
python_module_name = "trainer.task"

PROJECT_ID = awesome-ridge-333421
BUCKET_NAME = gs://story-gen2
REGION = us-central1


In [5]:
!gsutil ls -l {python_package_gcs_uri}

      4355  2021-12-03T07:15:54Z  gs://story-gen2/story2/python_package/dist/trainer-0.1.tar.gz
TOTAL: 1 objects, 4355 bytes (4.25 KiB)


In [8]:
!gsutil cp -r gs://story-gen2/story2/custom_trainer ./
!gsutil cp -r gs://story-gen2/story2/python_package ./

Copying gs://story-gen2/story2/custom_trainer/Dockerfile...
Copying gs://story-gen2/story2/custom_trainer/scripts/train-cloud.sh...         
/ [2 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 2 objects/2.9 KiB.                                      
Copying gs://story-gen2/story2/python_package/dist/trainer-0.1.tar.gz...
Copying gs://story-gen2/story2/python_package/scripts/train-cloud.sh...         
Copying gs://story-gen2/story2/python_package/setup.py...                       
Copying gs://story-gen2/story2/python_package/trainer.egg-info/PKG-INFO...      
/ [4 files][  7.1 KiB/  7.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://story-gen2/story2/python_package/t

In [9]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 -m trainer.task

Namespace(adam_epsilon=1e-08, batch_size=4, job_dir='gs://output-story/models/story-gen', learning_rate=1e-05, model_name='story-gen', num_epochs=1, warmup=0.1, weight_decay=0.01)
loading data
Downloading: 100%|█████████████████████████| 0.99M/0.99M [00:00<00:00, 4.89MB/s]
Downloading: 100%|███████████████████████████| 446k/446k [00:00<00:00, 2.78MB/s]
Downloading: 100%|█████████████████████████| 1.29M/1.29M [00:00<00:00, 6.66MB/s]
Downloading: 100%|██████████████████████████████| 665/665 [00:00<00:00, 371kB/s]
loading data completed
Downloading: 100%|███████████████████████████| 523M/523M [00:07<00:00, 70.0MB/s]
***** Running training *****
  Num examples = 12
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3
100%|█████████████████████████████████████████████| 3/3 [00:50<00:00, 16.71s/it]***** Running Evaluation *****
  Num examples = 12
  Ba

In [5]:
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt

print(
    f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI}"
)
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"python_module_name={python_module_name}")
JOB_NAME = f"story-gen-pytorch-pkg-ar-{get_timestamp()}"
DISPLAY_NAME=f"story-gen-{get_timestamp()}"
print(f"JOB_NAME={JOB_NAME}")
print(f"JOB_NAME={DISPLAY_NAME}")

PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/training/pytorch-xla.1-7:latest
python_package_gcs_uri=gs://story-gen2/story2/python_package/dist/trainer-0.1.tar.gz
python_module_name=trainer.task
JOB_NAME=story-gen-pytorch-pkg-ar-20211204165936
JOB_NAME=story-gen-20211204165936


In [6]:
aiplatform.init(project=PROJECT_ID, location=REGION,staging_bucket='output-story')
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"{JOB_NAME}",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
    model_serving_container_image_uri=deploy_image
)

In [36]:
training_args = ["--num_epochs", "1", "--model_name", "story-gen","--job_dir","/output-story/models"]

model = job.run(
    model_display_name=DISPLAY_NAME,
    replica_count=1,
    machine_type="n1-standard-8",
    args=training_args,
    base_output_dir=f'gs://output-story/models/story-gen',
    sync=False,
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://output-story/models/story-gen 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4218783234823028736?project=60899115109
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/60899115109/locations/us-central1/trainingPipelines/4218783234823028736 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5465822934767501312?project=60899115109
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/60899115109/locations/us-central1/trainingPipelines/4218783234823028736 current state:
PipelineState.PIPELINE_STATE_RUNNING


In [79]:
%%writefile ./predictor/custom_text_handler.py

import os
import json
import logging
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from ts.torch_handler.base_handler import BaseHandler
from transformers import GPT2Tokenizer, GPT2LMHeadModel

logger = logging.getLogger(__name__)


class MyHandler(BaseHandler):
    """
    The handler takes an input string and returns the classification text 
    based on the serialized transformers checkpoint.
    """
    def __init__(self):
        super().__init__()
        self.initialized = False

    def initialize(self,ctx):
        """ Loads the model.pt file and initialized the model object.
        Instantiates Tokenizer for preprocessor to use
        Loads labels to name mapping file for post-processing inference response
        """    
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        serialized_file = self.manifest["model"]["serializedFile"]
        model_pt_path = os.path.join(model_dir, serialized_file)
        if not os.path.isfile(model_pt_path):
            raise RuntimeError("Missing the model.pt or pytorch_model.bin file")
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")
        # Load model
        self.model = GPT2LMHeadModel.from_pretrained(model_dir)
        print('loading model and tokenizer')
        # Ensure to use the same tokenizer used during training

        self.initialized = True

    def preprocess(self, data):
        """ Preprocessing input request by tokenizing
            Extend with your own preprocessing steps as needed
        """
        print("preprocess")
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")
        print("text is",text)
        s = text.decode('utf-8')
        logger.info("Received text: '%s'", s)
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        tokenizer.pad_token=tokenizer.eos_token
        # Tokenize the texts
        for p in '!,.:;?':
            s=s.replace(' '+p,p)
            s=s.replace(' '+'n\'t','n\'t')
            s=s.replace(' '+'\'s','\'s')
            s=s.replace(' '+'\'re','\'re')
            s=s.replace(' '+'\'ve','\'ve')
            s=s.replace(' '+'\'ll','\'ll')
            s=s.replace(' '+'\'am','\'am')
            s=s.replace(' '+'\'m','\'m')
            s=s.replace(' '+'\' m','\'m')
            s=s.replace(' '+'\'m','\'m')
            s=s.replace(' '+'\' ve','\'ve')
            s=s.replace(' '+'\' s','\'s')
            s=s.replace('<newline>','\n')
        
        t=list(s)
        logger.info("sentence: '%s'",s)
        tokenizer_args = ((s,))
        logger.info("tokenizer args: %s", *tokenizer_args)
        encoded_prompt = tokenizer.encode(*tokenizer_args, add_special_tokens=False, return_tensors="pt")
        print("preprocess done")
        print(encoded_prompt)
        return encoded_prompt

    def inference(self, inputs):
        """ Predict the class of a text using a trained transformer model.
        """
        print('inference')
        print(inputs)
        output_sequences = self.model.generate(
        input_ids=inputs,
        max_length=300,
        temperature=1,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=3
        )
        if len(output_sequences.shape) > 2:
            output_sequences.squeeze_()
        texts=[]
        print('inference end')
        for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
            print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
            generated_sequence = generated_sequence.tolist()
            # Decode text
            tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            tokenizer.pad_token=tokenizer.eos_token
            text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
            # Remove all text after eos token
            text = text[: text.find(tokenizer.eos_token)]
            texts.append(text)
            print(text)
        
        return [texts]
        

    def postprocess(self, inference_output):
        print('post')
        return inference_output
    # def handle(self, data, ctx):
    #     model_input = self.preprocess(data)
    #     model_output = self.inference(model_input)
    #     return self.postprocess(model_output)
        

Overwriting ./predictor/custom_text_handler.py


In [61]:
GCS_MODEL_ARTIFACTS_URI = "gs://output-story/models/story-gen/story-gen"
APP_NAME='story-gen'
CUSTOM_PREDICTOR_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(f"CUSTOM_PREDICTOR_IMAGE_URI = {CUSTOM_PREDICTOR_IMAGE_URI}")
!gsutil cp -r $GCS_MODEL_ARTIFACTS_URI ./predictor

CUSTOM_PREDICTOR_IMAGE_URI = gcr.io/awesome-ridge-333421/pytorch_predict_story-gen
Copying gs://output-story/models/story-gen/story-gen/config.json...
Copying gs://output-story/models/story-gen/story-gen/pytorch_model.bin...       
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://output-story/models/story-gen/story-gen/training_args.bin...       
| [3 files][486.8 MiB/486.8 MiB]                                                
Operation completed over 3 objects/486.8 MiB.                                    


In [80]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./predictor/Dockerfile

FROM pytorch/torchserve:latest

# install dependencies
RUN pip3 install transformers
# RUN git clone https://github.com/pytorch/serve.git
# RUN pip3 install torchserve torch-model-archiver torch-workflow-archiver
RUN pip3 install torch torchvision torchserve torch-model-archiver -f https://download.pytorch.org/whl/torch_stable.html

# copy model artifacts, custom handler and other dependencies
# RUN pwd
# RUN ls -lart /home
COPY ./custom_text_handler.py /home/model-server/
COPY ./story-gen/ /home/model-server/

# create torchserve configuration file
USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:9000" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:9001" >> /home/model-server/config.properties
RUN printf "\nmetrics_address=http://0.0.0.0:9002" >> /home/model-server/config.properties
# RUN printf "\ninstall_py_dep_per_model=true" >> /home/model-server/config.properties

USER model-server

# expose health and prediction listener ports from the image
EXPOSE 9000
EXPOSE 9001

# create model archive file packaging model artifacts and dependencies
RUN torch-model-archiver -f \
  --model-name=$APP_NAME \
  --version=1.0 \
  --serialized-file=/home/model-server/pytorch_model.bin \
  --handler=/home/model-server/custom_text_handler.py \
  --extra-files "/home/model-server/config.json,/home/model-server/training_args.bin" \
  --export-path=/home/model-server/model-store

# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \
     "--start", \
     "--ncs", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "$APP_NAME=$APP_NAME.mar", \
     "--model-store", \
     "/home/model-server/model-store"]

EOF

echo "Writing ./predictor/Dockerfile"

Writing ./predictor/Dockerfile


In [81]:
!docker build \
  --tag=$CUSTOM_PREDICTOR_IMAGE_URI \
  ./predictor

Sending build context to Docker daemon  1.021GB
Step 1/15 : FROM pytorch/torchserve:latest
 ---> 84cd5f2ee48d
Step 2/15 : RUN pip3 install transformers
 ---> Using cache
 ---> 621300b0465a
Step 3/15 : RUN pip3 install torch torchvision torchserve torch-model-archiver -f https://download.pytorch.org/whl/torch_stable.html
 ---> Using cache
 ---> 85a79afe2649
Step 4/15 : COPY ./custom_text_handler.py /home/model-server/
 ---> cdc1e620d391
Step 5/15 : COPY ./story-gen/ /home/model-server/
 ---> a8eaa31566c9
Step 6/15 : USER root
 ---> Running in 91eff801029b
Removing intermediate container 91eff801029b
 ---> 66002ae053f8
Step 7/15 : RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
 ---> Running in e55116755760
Removing intermediate container e55116755760
 ---> 8514e9bf4755
Step 8/15 : RUN printf "\ninference_address=http://0.0.0.0:9000" >> /home/model-server/config.properties
 ---> Running in 0b8c2502f043
Removing intermediate container 0b8c2502f043
 ---> a8118e

In [82]:
!docker stop story
!docker run -t -d --rm -p 9000:9000 --name=story $CUSTOM_PREDICTOR_IMAGE_URI
!sleep 20

story
1d65f5f16a0bab3106ddeb6728abca5df6fb04c4a4162cf996d67d5f2e5c43a5


In [83]:
!curl http://localhost:9000/ping

{
  "status": "Healthy"
}


In [84]:
%%bash -s $APP_NAME

APP_NAME=$1

cat > ./predictor/instances.json <<END
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo 'Childrens logic dictates the way the world works.' | base64 --wrap=0)"
       }
     }
   ]
}   

END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./predictor/instances.json \
  http://localhost:9000/predictions/$APP_NAME/

{"predictions": [["Childrens logic dictates the way the world works.\n\nWas there any sort of intentionally faked story to tease the audience at age 12, but the boys sure looked like they belonged in the movies. Suddenly, a kid woke up one morning, pushed a baby out of the little dirt and let her ride down the same section of street, crying out and claiming, \"I'm little.\"\n\nThat certainly wasn't the only time the voice in the corner of his mind shouted out: \"Where's the chicken?\" His roommate had a pet cat when she was 11. But eventually, after the gentle pet calmed him down, she told him she heard her older sister crying and stomping over some nearby trees, thinking: \"I was crying! I'm fucking making my grandpa cry!\"\n\nWithout a break, he left his couch and walked to the driveway. It was around 3:30 p.m. When he returned to school, he saw the little cat not at school, but at a place the guys loved: a dusty place in front of one of the shiny, white-on-a-car windows. He figured 

In [85]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/awesome-ridge-333421/pytorch_predict_story-gen]

dba05798: Preparing 
ad5587fd: Preparing 
970ea338: Preparing 
6539f803: Preparing 
e299204a: Preparing 
1aea82dd: Preparing 
07b07f5b: Preparing 
0a3aa641: Preparing 
faf96803: Preparing 
bf18a086: Preparing 
8fc6c1fc: Preparing 
8846dbe4: Preparing 
411b72c6: Preparing 
b1b63053: Preparing 
aea82dd: Pushed   510.4MB/510.4MBlatest: digest: sha256:02e801b28d66442ead54914aa7836d90cc217a3d1edab183fd137e46f0e00625 size: 3665


In [86]:
model_display_name = f"{APP_NAME}-{get_timestamp()}"
model_description = "PyTorch based story generation with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [9000]

In [87]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/60899115109/locations/us-central1/models/3240555436172115968/operations/8020228208345481216
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/60899115109/locations/us-central1/models/3240555436172115968
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/60899115109/locations/us-central1/models/3240555436172115968')
story-gen-20211204033549
projects/60899115109/locations/us-central1/models/3240555436172115968


In [88]:
endpoint_display_name = f"{APP_NAME}-endpoint-{get_timestamp()}"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)
traffic_percentage = 100
machine_type = "n1-standard-4"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/60899115109/locations/us-central1/endpoints/8203174779860025344/operations/9220437509039718400
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/60899115109/locations/us-central1/endpoints/8203174779860025344
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/60899115109/locations/us-central1/endpoints/8203174779860025344')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/60899115109/locations/us-central1/endpoints/8203174779860025344
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/60899115109/locations/us-central1/endpoints/8203174779860025344/operations/2282642283075469312
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/60899115109/loca

resource name: projects/60899115109/locations/us-central1/endpoints/8203174779860025344

In [14]:
filter = f'display_name="{endpoint_display_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=filter):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)



In [26]:
endpoint = aiplatform.Endpoint('projects/60899115109/locations/us-central1/endpoints/8203174779860025344')

In [27]:
test_instances = [
    b"Childrens logic dictates the way the world works."
]

In [29]:
import base64
import json
print("=" * 100)
for instance in test_instances:
    print(f"Input text: \n\t{instance.decode('utf-8')}\n")
    b64_encoded = base64.b64encode(instance)
    test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    print(f"Formatted input: \n{json.dumps(test_instance, indent=4)}\n")
    prediction = endpoint.predict(instances=test_instance)
    print(f"Prediction response: \n\t{prediction}")
    # for i in range(0,2):
    #     print(f'GENERATED SEQUENCE {i+1}\n')
    #     print(f'{prediction[0][0][i]}\n')
    print("=" * 100)

Input text: 
	Childrens logic dictates the way the world works.

Formatted input: 
[
    {
        "data": {
            "b64": "Q2hpbGRyZW5zIGxvZ2ljIGRpY3RhdGVzIHRoZSB3YXkgdGhlIHdvcmxkIHdvcmtzLg=="
        }
    }
]

Prediction response: 
	Prediction(predictions=[['Childrens logic dictates the way the world works. Unrighteous kids are taught that if they fail to pay their bills, you shouldn\'t be bothered by school officials; they have to deal with bullies in real time. I can remember seeing a line on my calendar while I was engaged in "Forgiving Dad." At any given moment I watched as my family\'s milk carton was being rammed into my face and my friends looked down at me.\n\nThough living in an Orthodox Christian family could have impacted how well our kids answered to this theology, the majority of the time the one that got away with failing to pay their bills eventually got in trouble. Meanwhile, when the next family, white and male, did, they simply decided to stay home and live ou

In [31]:
for i in range(0,3):
    print(f'GENERATED SEQUENCE {i+1}\n')
    print(f'{prediction[0][0][i]}\n')

GENERATED SEQUENCE 1

Childrens logic dictates the way the world works. Unrighteous kids are taught that if they fail to pay their bills, you shouldn't be bothered by school officials; they have to deal with bullies in real time. I can remember seeing a line on my calendar while I was engaged in "Forgiving Dad." At any given moment I watched as my family's milk carton was being rammed into my face and my friends looked down at me.

Though living in an Orthodox Christian family could have impacted how well our kids answered to this theology, the majority of the time the one that got away with failing to pay their bills eventually got in trouble. Meanwhile, when the next family, white and male, did, they simply decided to stay home and live outside the house.

In the classroom, many students read my Bible each day. If I asked someone "what do you mean 'practicing Jehovah's Witnesses'?" they invariably would reply, "because I didn't want to ask." Most Americans are in no position to publi